In [2]:
from IPython.display import display, HTML
display(HTML("""<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{fontsize:12pt;padding:5px;}
table.dataframe{font-size:12px;}))
</style>
"""))

# <span style="color:red">ch09 08 chroma → pinecorn을 활용한 RAG구현 </span>

# 벡터 DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB

# 0. 패키지 설치

In [2]:
%pip install -q pinecone-client langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# 1. Knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)

183

In [1]:
# embedding : openAI API text-embedding-3-large

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model = "text-embedding-3-large")

In [3]:
%%time

# pinecone vector database

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index"
# database = PineconeVectorStore.from_documents(
#     documents = document_list,
#     embedding = embedding,
#     index_name = index_name
# )

database = PineconeVectorStore(
    embedding = embedding,  # 질문을 임베딩하여 유사도 검색
    index_name = index_name
)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 547 ms
Wall time: 1.96 s


# 2. 답변 생성을 위한 Retrieval

In [4]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.as_retriever(
    # search_kwargs = {'k':4}
)
retriever.invoke(query)

[Document(id='f91f82cf-a379-45a4-812d-abfd1a2e98b8', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(

# 3. 제공되는 prompt를 활용하여 답변 생성

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4.1-nano")

In [6]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,  # database.as_retriever()
    chain_type_kwargs = {"prompt":prompt}
)

In [7]:
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '기본적으로 연봉 5천만원인 직장인의 소득세는 정확히 계산하려면 여러 공제와 세율이 필요하나, 일반적으로 2000만~3000만원 정도의 세액이 예상됩니다. 구체적인 세금액은 공제액, 세율, 세액공제, 기타 소득공제 요인에 따라 달라집니다. 따라서 정확한 금액을 알기 위해서는 세무서의 세율표와 공제 항목을 참고해야 합니다.'}